In [178]:
#%%writefile Si570.py
class Si570( object ):
    """ 
    helper class to convert from numbers used in the datasheet
    to register values
    """
    HS_DIVS_LOOKUP = (4,5,6,7,None,9,None,11)
    
    def __init__(self, l=None):
        """
        l can be a string like 'r 01 C2 BC 81 83 02 \n'
        """
        if l:
            ls = l.strip().split()[1:] # ['01', 'C2', 'BC', '81', '83', '02']
            self._regs = bytearray([ int(x,16) for x in ls ])
        else:
            self._regs = bytearray(6);
    
    @property
    def HS_DIV(self):
        """ DCO High Speed Divider """
        return Si570.HS_DIVS_LOOKUP[ self._regs[0] >> 5 ]
    
    @HS_DIV.setter
    def HS_DIV(self, value):
        ind = Si570.HS_DIVS_LOOKUP.index(value)
        self._regs[0] &= 0x1F
        self._regs[0] |= (ind<<5) & 0xE0
    
    @property
    def N1(self):
        """ CLKOUT Output Divider """
        N1 = (self._regs[0]&0x1F)<<2 | self._regs[1]>>6
        N1 += 1
        # Illegal odd divider values will be rounded up to the nearest even value.
        if( (N1 > 1) and (N1 & 0x01) ):
            print( "Illegal N1: {0}, rounding up to {1}", N1, N1+1 )
            N1 += 1
        return N1
    
    @N1.setter
    def N1(self, value):
        value -= 1
        self._regs[0] &= 0xE0
        self._regs[0] |= (value>>2) & 0x1F
        self._regs[1] &= 0x3F
        self._regs[1] |= (value<<6) & 0xC0
    
    @property
    def RFFREQ(self):
        """ Reference Frequency control input to DCO. [float] """
        RFFREQ = (self._regs[1]&0x3F)<<32 | self._regs[2]<<24 |         \
                        self._regs[3]<<16 | self._regs[4]<<8  | self._regs[5]
        return RFFREQ / 2.0**28
    
    @RFFREQ.setter
    def RFFREQ(self, value):
        value = int( value * 2**28 )
        self._regs[1] &= 0xC0
        self._regs[1] |= (value>>32) & 0x3F
        self._regs[2]  = (value>>24) & 0xFF
        self._regs[3]  = (value>>16) & 0xFF
        self._regs[4]  = (value>> 8) & 0xFF
        self._regs[5]  =  value      & 0xFF
    
    def fxtal(self, f0):
        """ 
        calculate internal crystal frequency. 
        f0 = startup frequency (see http://www.silabs.com/products/timing/lookup-customize)
        """
        return ( f0 * self.HS_DIV * self.N1 ) / self.RFFREQ
    
    def __repr__(self):
        s  = 'w '
        s += ' '.join('{:02x}'.format(x) for x in self._regs)
        return s
    
    def __str__(self):
        s = "HS_DIV:{0:2d}, N1:{1:2d}, RFFREQ:{2:13.9f}".format( self.HS_DIV, self.N1, self.RFFREQ )
        return s

In [179]:
l = "r 01 C2 BC 81 83 02 \n"
s = Si570( l )
print( s )
print( s.HS_DIV )
print( s.N1 )
print( s.RFFREQ )
print( s.fxtal( 156e6 )/1e6 )

HS_DIV: 4, N1: 8, RFFREQ: 43.781619079
4
8
43.78161907941103
114.02045207477408


In [158]:
print( l.encode("ascii") )

b'r 01 C2 BC 81 83 02 \n'


In [111]:
s.RFFREQ = s.RFFREQ
s.HS_DIV = s.HS_DIV
s.N1 = s.N1

In [112]:
print( s._regs )
print( s.HS_DIV )
print( s.N1 )
print( s.RFFREQ )
print( s.fxtal( 156e6 )/1e6 )

bytearray(b'\x01\xc2\x80\x00\x00\x00')
4
8
40.0
124.8


In [159]:
import serial

In [167]:
ser = serial.Serial("/dev/ttyUSB0", timeout=3 )

In [172]:
ser.write(b"w 01 c2 eb 34 d0 02")
print( ser.readline().decode().strip() )
print( ser.readline().decode().strip() )

w 01 C2 EB 34 D0 02
config_done


In [165]:
ser.close()